# Wöhler analyzing tool
Developed by Mustapha Kassem in scope of a master thesis at TU München

## Pylife Woehler-curve evaluation script

### Initialization

In [1]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy import stats, optimize
import mystic as my
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from os import path
import io
import sys, os
import json

sys.path.insert(0, os.path.abspath('..'))

from pylife.materialdata.woehler.fatigue_data import FatigueData
from pylife.materialdata.woehler.woehler_curve_creator import WoehlerCurveCreator
from pylife.materialdata.woehler.probability_plot_creator import ProbabilityPlotCreator
from pylife.materialdata.woehler.woehler_curve_diagrams import WoehlerCurveDiagrams
from pylife.materialdata.woehler.plot_probability_plot_finite import PlotProbabilityPlotFinite
from pylife.materialdata.woehler.plot_probability_plot_infinite import PlotProbabilityPlotInfinite
from pylife.materialdata.woehler.woehler_curve_widgets import WoehlerCurveWidgets
from pylife.materialdata.woehler.data_file_display import DataFileDisplay
from pylife.materialdata.woehler.woehler_curve_creator_options import  WoehlerCurveCreatorOptions
from pylife.materialdata.woehler.woehler_curve_data_plotter import  WoehlerCurveDataPlotter
from pylife.materialdata.woehler.whole_woehler_curve_plotter import  WholeWoehlerCurvePlotter

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import warnings
import pdb

### Data import
#### Data is made up of two columns:
 * The first column is made up of the load values
 * The scond column is made up of the load-cycle values
 

In [3]:
file_name = 'woehler-test-data.csv'


##### Transforming data in excel to python arrays

In [4]:
data = pd.read_csv(file_name, sep='\t')
data.columns=['loads', 'cycles']
rb = DataFileDisplay(data)

##### For automization purposes
data = pd.read_excel('../data/Test_dat.xlsx')
data.columns=['loads', 'cycles']

#### 2. Enter the load cycle limit (The load-cycle value that seperates Fractures from Run-outs):
##### Note: in case the load cycle limit is the highest number found in the column set it to *max(data[:,1])*

In [5]:
ld_cyc_lim = data['cycles'].max()
#ld_cyc_lim = 10e6

## Parameters

#### 3. Fill in the fixed parameter values. 
#### The empty parameters are the ones to be estimated.
##### Note: don't rerun the cell after typing the values

In [6]:
fatigue_data = FatigueData(data, ld_cyc_lim)
woehler_curve_creator_options = WoehlerCurveCreatorOptions(fatigue_data)

In [7]:
woehler_curve_widgets = WoehlerCurveWidgets()

#### 4. Variable definition of class WoehlerCurve:

## Visualization of Results
#### 5. Enter necessary description to personalize the diagram:
 * For a default diagram description set the value default = 1
 * Enter the units for the axes as a string variable [amp, ld_typ, unit]

##### x- and y-axis are dynamically scaled to fit the data in a diagram. If the User wishes to change them:
 * Enter the limits for the x- and y-axis [xlim_WL and ylim_WL]
 <br>
(example: xlim_WL = (4E5, 8E10) )

In [8]:
woehler_curve_data_plotter = WoehlerCurveDataPlotter(woehler_curve_creator_options.woehler_curve)

#### 6. Choose the plot type to be visualized in the following cell

In [9]:
whole_woeler_curve_plotter = WholeWoehlerCurvePlotter(woehler_curve_creator_options.woehler_curve)

In [10]:
w4 = woehler_curve_widgets.results_visual_probability_plot()
print('Choose a plot type and execute the following cell to visualize the results:')
w4

Choose a plot type and execute the following cell to visualize the results:


RadioButtons(description='Plot Type', options=(('Probability plot of the finite zone', 'Probability plot of th…

In [11]:
probability_plot_creator = ProbabilityPlotCreator(fatigue_data)
finite_probability_plot = probability_plot_creator.probability_plot_finite()
infinite_probability_plot = probability_plot_creator.probability_plot_infinite()
plot_probability_plot_finite = PlotProbabilityPlotFinite(finite_probability_plot)
plot_probability_plot_finite.plot_probability_plot()
plot_probabilty_plot_infinite = PlotProbabilityPlotInfinite(infinite_probability_plot)
plot_probabilty_plot_infinite.plot_probability_plot()

### Final Woehler-curve plot
#### 7. Plot of the complete Woehler curve.
#### Choose the value of $k_2$ to plot the figure.

In [ ]:
k_1 = woehler_curve.curve_parameters['k_1']
w5 = WoehlerWidget.inf_plot(woehler_curve, k_1)
print('Choose a value for k_2:')
w5

### 8. Choose the parameters to plot
- Probit: $k_{regression}$, $\frac{1}{T_N}$, $ND_{50}$, $SD_{50}$(Probit),$\frac{1}{T_S}$(Probit)
- Mali: Maximum likelihood parameters (including the parameters fixed by the user)

In [ ]:
w2 = WoehlerWidget.method_mali_probit()
w2

In [ ]:
if len(woehler_curve.fatigue_data.ld_lvls_inf[0])<2 and w2.value == "Probit":
    print("Warning:\n")
    print("Not enough load levels in the infinite zone for the probit method, please choose the Mali")
    print("\nparameters for visualization")
else:
    SD50, ND50, TS = PlotWC.runout_zone_method(woehler_curve, method=w2.value, slope_chosen=w5.value)
    _ = PlotWC.final_curve_plot(woehler_curve, SD50, ND50, TS, w5.value, w2.value, amp, ld_typ, unit, xlim_WL, ylim_WL, default)

## BIC
#### Bayesian Information Criterion: is a criterion for model selection among a finite set of models;
#### the model with the lowest BIC is preferred.

In [ ]:
sum_lolli = woehler_curve_creator.mali_sum_lolli(woehler_curve.curve_parameters['SD_50'], woehler_curve.curve_parameters['1/TS'], 
                                        woehler_curve.curve_parameters['k_1'], woehler_curve.curve_parameters['ND_50'], 
                                        woehler_curve.curve_parameters['1/TN'], woehler_curve.fatigue_data.fractures,
                                        woehler_curve.fatigue_data.zone_inf, woehler_curve.fatigue_data.load_cycle_limit)

n_data_pt = data.shape[0]
bic = WC_data.bayes_inf_crit(sum_lolli, WC_data.p_opt, n_data_pt)
print('\nBIC =',bic)

## Export variable data

In [ ]:
with open('param_dict.json', 'w') as fp:
    json.dump({'Mali_%dp_result_estimated'%len(WC_data.p_opt):estim_param, 
               'Mali_%dp_result_fixed'%len(WC_data.p_opt):fixed_param,
               'Mali_2p_result':WC_data.Mali_2p_result,
               'Probit_result':WC_data.Probit_result, 'k_reg': WC_data.k, '1/TN prl-chn':WC_data.TN,
               'BIC':bic}, fp)

## Reload variable data

In [ ]:
with open('param_dict.json', 'r') as fp:
    dict_import = json.load(fp)
dict_import